# 🤖 Análise de Consenso entre LLMs
## Notebook Refatorado

Este notebook usa a estrutura refatorada com:
- Componentes modulares
- Logging com loguru
- Integração com HuggingFace

## 1️⃣ Setup e Configuração

In [2]:
# Imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger

# Configurar logging
logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

# Paths
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root / 'src' / 'llm_annotation_system'))
sys.path.insert(0, str(project_root / 'src' / 'config'))
sys.path.insert(0, str(project_root / 'src' / 'utils'))

logger.success("✓ Setup completo")

14:59:29 | SUCCESS  | ✓ Setup completo


## 2️⃣ Carregar Dataset do HuggingFace

In [ ]:
from src.utils.data_loader import load_hf_dataset, list_available_datasets

# Listar datasets
logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

ModuleNotFoundError: No module named 'data_loader'

In [ ]:
# Carregar dataset
dataset_name = "agnews"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(dataset_name)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

In [ ]:
# Visualizar amostra
logger.info("\nAmostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

## 3️⃣ Configurar Modelos LLM

In [ ]:
from llm_annotation_system.annotation.llm_annotator import LLMAnnotator

# Modelos (ajuste conforme disponível)
models = [
    "llama3-8b",
    "mistral-7b",
    "qwen2-7b",
]

# Inicializar
annotator = LLMAnnotator(
    models=models,
    categories=categories,
    api_keys=None,  # Ou configure com suas keys
    use_langchain_cache=True
)

logger.success("✓ Annotator inicializado")

## 4️⃣ Executar Anotação

In [ ]:
# Parâmetros
num_repetitions = 3

# Estimativa
total_annotations = len(texts) * len(models) * num_repetitions
logger.info(f"Total de anotações: {total_annotations}")

# Anotar
df_annotations = annotator.annotate_dataset(
    texts=texts,
    num_repetitions=num_repetitions
)

logger.success("✓ Anotações completas")
display(df_annotations.head())

## 5️⃣ Calcular Consenso

In [ ]:
# Calcular consenso
df_with_consensus = annotator.calculate_consensus(df_annotations)

# Estatísticas
logger.info("\n📊 Estatísticas de Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Mediana: {df_with_consensus['consensus_score'].median():.2%}")
logger.info(f"  Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

# Distribuição por nível
levels = df_with_consensus['consensus_level'].value_counts()
logger.info("\nDistribuição por nível:")
for level, count in levels.items():
    logger.info(f"  {level}: {count} ({count/len(df_with_consensus):.1%})")

In [ ]:
# Visualizações
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(df_with_consensus['consensus_score'], bins=20, edgecolor='black')
axes[0].set_xlabel('Consensus Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribuição de Scores de Consenso')

# Barras por nível
levels.plot(kind='bar', ax=axes[1], color=['green', 'orange', 'red'])
axes[1].set_xlabel('Nível de Consenso')
axes[1].set_ylabel('Contagem')
axes[1].set_title('Casos por Nível de Consenso')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 6️⃣ Análise Detalhada

In [ ]:
from consensus_analyzer_refactored import ConsensusAnalyzer

# Inicializar analyzer
analyzer = ConsensusAnalyzer(categories)

# Colunas de consenso
consensus_cols = [col for col in df_with_consensus.columns if '_consensus' in col and '_score' not in col]

# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus,
    annotator_cols=consensus_cols,
    output_dir="./results"
)

logger.success("✓ Relatório gerado")

In [ ]:
# Métricas
logger.info("\n📊 Métricas de Distância:")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

# Interpretação
kappa = report['fleiss_kappa']
if kappa > 0.8:
    logger.success("Concordância excelente!")
elif kappa > 0.6:
    logger.info("Concordância boa")
elif kappa > 0.4:
    logger.warning("Concordância moderada")
else:
    logger.warning("Concordância fraca")

In [ ]:
# Matriz de concordância
agreement_df = report['pairwise_agreement']

plt.figure(figsize=(10, 8))
sns.heatmap(agreement_df, annot=True, fmt='.2f', cmap='YlGnBu', cbar_kws={'label': 'Agreement'})
plt.title('Matriz de Concordância Par a Par')
plt.tight_layout()
plt.show()

In [ ]:
# Casos problemáticos
problematic = report.get('problematic_cases')
if problematic is not None and len(problematic) > 0:
    logger.warning(f"\n⚠️  {len(problematic)} casos problemáticos identificados")
    display(problematic.head())
else:
    logger.success("\n✓ Nenhum caso problemático identificado")

## 7️⃣ Validação com Ground Truth

In [ ]:
if ground_truth:
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    # Adicionar ground truth
    df_with_consensus['ground_truth'] = ground_truth
    
    # Accuracy
    accuracy = accuracy_score(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    logger.success(f"\n🎯 Accuracy: {accuracy:.2%}")
    
    # Classification report
    logger.info("\nClassification Report:")
    print(classification_report(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    ))
    
    # Confusion matrix
    cm = confusion_matrix(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix vs Ground Truth')
    plt.tight_layout()
    plt.savefig('./results/confusion_vs_ground_truth.png', dpi=150)
    plt.show()
    
    # Accuracy por nível de consenso
    logger.info("\nAccuracy por nível de consenso:")
    for level in ['high', 'medium', 'low']:
        df_level = df_with_consensus[df_with_consensus['consensus_level'] == level]
        if len(df_level) > 0:
            acc_level = accuracy_score(
                df_level['ground_truth'],
                df_level['most_common_annotation']
            )
            logger.info(f"  {level}: {acc_level:.2%} ({len(df_level)} casos)")
else:
    logger.info("\n⚠️  Ground truth não disponível - pulando validação")

## 8️⃣ Exportar Resultados

In [ ]:
import json
from pathlib import Path

# Criar diretório
results_dir = Path('./results/final')
results_dir.mkdir(parents=True, exist_ok=True)

# Salvar CSVs
df_with_consensus.to_csv(results_dir / 'dataset_anotado_completo.csv', index=False)
logger.info(f"✓ Salvos: {len(df_with_consensus)} registros")

# Alta confiança
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf.to_csv(results_dir / 'alta_confianca.csv', index=False)
logger.info(f"✓ Alta confiança: {len(high_conf)} registros")

# Necessita revisão
low_conf = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
low_conf.to_csv(results_dir / 'necessita_revisao.csv', index=False)
logger.info(f"✓ Necessita revisão: {len(low_conf)} registros")

# Sumário JSON
summary = {
    'dataset': {
        'name': dataset_name,
        'total_texts': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None
    },
    'config': {
        'models': models,
        'num_repetitions': num_repetitions,
        'total_annotations': total_annotations
    },
    'results': {
        'consensus_mean': float(df_with_consensus['consensus_score'].mean()),
        'consensus_median': float(df_with_consensus['consensus_score'].median()),
        'consensus_std': float(df_with_consensus['consensus_score'].std()),
        'high_consensus': int((df_with_consensus['consensus_level'] == 'high').sum()),
        'medium_consensus': int((df_with_consensus['consensus_level'] == 'medium').sum()),
        'low_consensus': int((df_with_consensus['consensus_level'] == 'low').sum()),
        'problematic': int(df_with_consensus['is_problematic'].sum())
    },
    'metrics': {
        'fleiss_kappa': float(report['fleiss_kappa']),
        'fleiss_interpretation': report['fleiss_interpretation']
    }
}

if ground_truth:
    summary['validation'] = {
        'accuracy': float(accuracy)
    }

with open(results_dir / 'sumario_experimento.json', 'w') as f:
    json.dump(summary, f, indent=2)

logger.success("\n✓ Resultados exportados com sucesso!")

## 9️⃣ Resumo Final

In [ ]:
logger.info("\n" + "="*80)
logger.success("RESUMO DO EXPERIMENTO")
logger.info("="*80)

logger.info(f"\n📊 Dataset: {dataset_name}")
logger.info(f"  Textos: {len(texts)}")
logger.info(f"  Categorias: {len(categories)}")

logger.info(f"\n🤖 Configuração:")
logger.info(f"  Modelos: {len(models)}")
logger.info(f"  Repetições: {num_repetitions}")
logger.info(f"  Total de anotações: {total_annotations}")

logger.info(f"\n📈 Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

if ground_truth:
    logger.info(f"\n🎯 Validação:")
    logger.info(f"  Accuracy: {accuracy:.2%}")

logger.info(f"\n📁 Arquivos gerados:")
logger.info(f"  {results_dir}/dataset_anotado_completo.csv")
logger.info(f"  {results_dir}/alta_confianca.csv")
logger.info(f"  {results_dir}/necessita_revisao.csv")
logger.info(f"  {results_dir}/sumario_experimento.json")

cache_stats = annotator.get_cache_stats()
logger.info(f"\n💾 Cache: {cache_stats['total_entries']} entradas")

logger.success("\n✅ Análise completa!")